In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="HZDrfvv1orCAdJ052SiY")
project = rf.workspace("universidadde-de-trsosmontes-e-alto-douro").project("rotten-apples-detection")
version = project.version(5)
dataset = version.download("multiclass")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Rotten-apples-detection-5 in multiclass:: 100%|██████████| 901/901 [00:00<00:00, 5914.32it/s]


In [6]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import os

In [7]:
# Setup basic paths
base_dir = '/content/Rotten-apples-detection-5'  # Replace with your path
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'valid')

In [8]:
# 1. Load and preprocess data
img_size = 224
batch_size = 16

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
!cd Rotten-apples-detection-5

In [11]:
%cd /content/Rotten-apples-detection-5/train

/content/Rotten-apples-detection-5/train


In [12]:
!ls

-00001_jpeg_jpg.rf.4b3d9c391f8301120d3b0dc72488d351.jpg
-00001_jpeg_jpg.rf.8ee5c56f83f0425298b7f548e4bb2801.jpg
-00001_jpeg_jpg.rf.cf8513f9463b75426614a3882bcff78f.jpg
-00002_jpeg_jpg.rf.56ac06d974d870d9750947868858d472.jpg
-00002_jpeg_jpg.rf.98be7c552902ab0f96dbf0bb7b979d6f.jpg
-00002_jpeg_jpg.rf.c90ea95c7bab0b1cadad95b4708200af.jpg
-00003_jpeg_jpg.rf.42f58b05a172e4d16dc4ed209411b9a1.jpg
-00003_jpeg_jpg.rf.46dc5b189309339e04df91312cfd3db3.jpg
-00003_jpeg_jpg.rf.51a69161c77a4c84fbcd86ed2eb3b6ce.jpg
-00003_jpeg_jpg.rf.75ada7490c0c8895597ddd3605dca342.jpg
-00003_jpeg_jpg.rf.dff6a2fc35017a8e902b10b2053ab724.jpg
-00003_jpeg_jpg.rf.f561625efeafbc12ef6d4bb87329ef91.jpg
-00006_jpeg_jpg.rf.0992936a8b9d9b30ee938a7b7b0d6bbf.jpg
-00006_jpeg_jpg.rf.24a13f2e9e1296b44606b1c984e31c59.jpg
-00006_jpeg_jpg.rf.5c9a74a6eb6b853e8e2c413421c96c50.jpg
-00007_jpeg_jpg.rf.04d56e72ecf788dbf50578aaabf2cf41.jpg
-00007_jpeg_jpg.rf.1d6d58dd068f122c6368d5eae5d13739.jpg
-00007_jpeg_jpg.rf.304bcc6bd4c5d8ef27c2c0a6292cd

In [13]:
!rm -rf 'Good Apple Rotten Apple'

In [14]:
train_gen = train_datagen.flow_from_directory(
    train_dir, target_size=(img_size, img_size), batch_size=batch_size, class_mode='categorical')

val_gen = val_datagen.flow_from_directory(
    val_dir, target_size=(img_size, img_size), batch_size=batch_size, class_mode='categorical')

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [15]:
# 2. Load VGG16 base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
base_model.trainable = False  # Freeze convolutional base

# 3. Add custom classification head
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_gen.num_classes, activation='sigmoid')(x)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [16]:
model = Model(inputs=base_model.input, outputs=predictions)

In [17]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 0)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,926,080 (68.38 MB)

 Trainable params: 3,211,392 (12.25 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [18]:
# 4. Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [19]:
# 5. Train model
history = model.fit(train_gen, epochs=5, validation_data=val_gen)

Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.


ValueError: The PyDataset has length 0

In [ ]:
# 6. Evaluate model
loss, acc = model.evaluate(val_gen)
print(f"Test Accuracy: {acc:.2f}")

In [ ]:
# 7. Inference: Predict and visualize some test images
class_labels = list(val_gen.class_indices.keys())

def plot_predictions(generator, model, num=5):
    for i in range(num):
        # Use the built-in next() function to get the next batch from the generator
        img, label = next(generator)
        pred = model.predict(img)
        plt.imshow(img[0])
        # Apply np.argmax to the label of the first image in the batch (label[0])
        plt.title(f"Predicted: {class_labels[np.argmax(pred[0])]}, Actual: {class_labels[np.argmax(label[0])]}")
        plt.axis('off')
        plt.show()

plot_predictions(val_gen, model, num=2)